In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
next_layer = "silver"

In [0]:
%run ../../../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
# Dynamic select query to read the data for specific proc_date
dynamic_query = f""" CREATE OR REPLACE TEMP VIEW temp_d_sto_header_spr  AS SELECT *
FROM udp_wcm_pro.udp_wcm_bronze_pro.supra_d_sto_header_spr   
WHERE proc_date like '{str(proc_date).replace('-', '')}%'
"""

# Execute the query
spark.sql(dynamic_query)
print(dynamic_query)

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_realtime.d_sto_header_spr

(
  hash_id BIGINT,
  sto_code STRING,
  rocket_code STRING,
  client_code STRING,
  wh_code STRING,
  wh_site_id STRING,
  external_code STRING,
  po_code STRING,
  promotion_code STRING,
  type STRING,
  request_type STRING,
  store_site_id STRING,
  store_name STRING,
  full_address STRING,
  addr_region STRING,
  addr_province STRING,
  addr_district STRING,
  addr_ward STRING,
  contact_name STRING,
  contact_phone STRING,
  created_date TIMESTAMP,
  canceled_date TIMESTAMP,
  cancellation_reason STRING,
  total_weight DOUBLE,
  total_volume DOUBLE,
  total_units DOUBLE,
  sto_picking_type STRING,
  condition_type STRING,
  source STRING,
  status STRING,
  calday TIMESTAMP,
  proc_date TIMESTAMP,
  reference_id STRING
) 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""   
CREATE OR REPLACE TEMPORARY VIEW view_d_sto_header_spr AS 
SELECT DISTINCT
    farm_fingerprint(CONCAT(
      COALESCE(CAST(sto_code AS STRING), ""),
      COALESCE(CAST(rocket_code AS STRING), ""),
      COALESCE(CAST(client_code AS STRING), ""),
      COALESCE(CAST(wh_code AS STRING), ""),
      COALESCE(CAST(wh_site_id AS STRING), ""),
      COALESCE(CAST(external_code AS STRING), ""),
      COALESCE(CAST(po_code AS STRING), ""),
      COALESCE(CAST(promotion_code AS STRING), ""),
      COALESCE(CAST(type AS STRING), ""),
      COALESCE(CAST(request_type AS STRING), ""),
      COALESCE(CAST(store_site_id AS STRING), ""),
      COALESCE(CAST(sto_picking_type AS STRING), ""),
      COALESCE(CAST(condition_type AS STRING), ""),
      COALESCE(CAST(source AS STRING), ""),
      COALESCE(CAST(status AS STRING), ""),
      COALESCE(CAST(TO_TIMESTAMP(calday, "yyyyMMddHHmmss") AS STRING), ""),
      COALESCE(CAST(TO_TIMESTAMP(proc_date, "yyyyMMddHHmmss") AS STRING), "")
    )) AS hash_id,
    CAST(sto_code AS STRING) AS sto_code,
    CAST(rocket_code AS STRING) AS rocket_code,
    CAST(client_code AS STRING) AS client_code,
    CAST(wh_code AS STRING) AS wh_code,
    CAST(wh_site_id AS STRING) AS wh_site_id,
    CAST(external_code AS STRING) AS external_code,
    CAST(po_code AS STRING) AS po_code,
    CAST(promotion_code AS STRING) AS promotion_code,
    CAST(type AS STRING) AS type,
    CAST(request_type AS STRING) AS request_type,
    CAST(store_site_id AS STRING) AS store_site_id,
    CAST(store_name AS STRING) AS store_name,
    CAST(full_address AS STRING) AS full_address,
    CAST(addr_region AS STRING) AS addr_region,
    CAST(addr_province AS STRING) AS addr_province,
    CAST(addr_district AS STRING) AS addr_district,
    CAST(addr_ward AS STRING) AS addr_ward,
    CAST(contact_name AS STRING) AS contact_name,
    CAST(contact_phone AS STRING) AS contact_phone,
    TO_TIMESTAMP(CAST(created_date AS STRING), "yyyyMMddHHmmss") AS created_date,
    TO_TIMESTAMP(CAST(canceled_date AS STRING), "yyyyMMddHHmmss") AS canceled_date,
    CAST(cancellation_reason AS STRING) AS cancellation_reason,
    CAST(total_weight AS DOUBLE) AS total_weight,
    CAST(total_volume AS DOUBLE) AS total_volume,
    CAST(total_units AS DOUBLE) AS total_units,
    CAST(sto_picking_type AS STRING) AS sto_picking_type,
    CAST(condition_type AS STRING) AS condition_type,
    CAST(source AS STRING) AS source,
    CAST(status AS STRING) AS status,
    TO_TIMESTAMP(calday, "yyyyMMddHHmmss") AS calday,
    TO_TIMESTAMP(proc_date, "yyyyMMddHHmmss") AS proc_date,
    CAST(reference_id AS STRING) AS reference_id
FROM temp_d_sto_header_spr;
  """)

In [0]:
spark.sql(f"""MERGE INTO {catalog_name}.udp_wcm_silver_realtime.d_sto_header_spr AS main_tbl
USING view_d_sto_header_spr AS temp_tbl 
ON main_tbl.hash_id = temp_tbl.hash_id
WHEN NOT MATCHED THEN 
  INSERT *;
  """)